# read images using pandas 

TODO:
1. Messidor, 2 classes, balanced
2. pre-processing

In [ ]:
from pathlib import Path
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np  # LINEAR ALGEBRA
import pandas as pd  # DATASET PREPROCESSOING : data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt  # VISULAIZATION : showing and rendering figures. Also used SEABORN
# io related
from skimage.io import imread
import os
from glob import glob
# not needed in Kaggle, but required in Jupyter
%matplotlib inline 


In [ ]:
base_image_dir = '../data/MESSIDOR_patches'
retina_df = pd.read_csv(os.path.join(base_image_dir, 'messidor_labels.csv'))
retina_df['level'] = retina_df['level'].map(lambda x: 0 if x == 0 else 1)
retina_df.tail()

In [ ]:
base_image_dir = '../data/MESSIDOR_patches'
retina_df = pd.read_csv(os.path.join(base_image_dir, 'messidor_labels.csv'))
retina_df['level2'] = retina_df['level'].map(lambda x: 0 if x == 0 else 1)
retina_df['PatientId'] = retina_df['image'] #map(lambda x: x.split('_')[0])
retina_df['path'] = retina_df['image'].map(lambda x: os.path.join(base_image_dir, '{}.jpg'.format(x)))
retina_df['exists'] = retina_df['path'].map(os.path.exists)
print(retina_df['exists'].sum(), 'images found of', retina_df.shape[0], 'total')

# retina_df['eye'] = retina_df['image'].map(lambda x: 1 if x.split('_')[-1] == 'left' else 0)
retina_df['level_cat'] = retina_df['level'].map(lambda x: to_categorical(x, 1+retina_df['level'].max()))
retina_df['level_cat2'] = retina_df['level2'].map(lambda x: to_categorical(x, 1+retina_df['level2'].max()))

retina_df.dropna(inplace=True)  # Remove missing values.
retina_df = retina_df[retina_df['exists']]
retina_df.sample(3)

In [ ]:
# retina_df[['level', 'eye']].hist(figsize=(10, 5))
retina_df[['level2']].hist(figsize=(10, 5))

In [ ]:
rr_df = retina_df[['PatientId', 'level2']].drop_duplicates()  # X=PatientId, y=level2
train_ids, valid_ids = train_test_split(rr_df['PatientId'], test_size=0.25, random_state=2018, stratify=rr_df['level2'])

# raw_train_df = retina_df[retina_df['PatientId'].isin(train_ids)]
# train_df = raw_train_df.groupby(['level', 'eye']).apply(lambda x: x.sample(75, replace = True)).reset_index(drop = True)
# train_df = raw_train_df.groupby(['level2']).apply(lambda x: x.sample(100, replace=True)).reset_index(drop=True)  # training sample

train_df = retina_df[retina_df['PatientId'].isin(train_ids)]  # training sample
valid_df = retina_df[retina_df['PatientId'].isin(valid_ids)]  # testing sample

In [ ]:
#print('train_df:', train_df.shape[0], 'raw_train_df:', raw_train_df.shape[0], 'valid_df:', valid_df.shape[0])

# train_df[['level', 'eye']].hist(figsize = (10, 5))
train_df[['level2']].hist(figsize=(10, 5))

In [ ]:
valid_df[['level2']].hist(figsize=(10, 5))

In [ ]:
y_train = np.array(train_df['level_cat2'])

y_train.shape

In [ ]:
y_train[2][0]

In [ ]:
train_df.tail()

In [ ]:
from PIL import Image

image_path = train_df['path'][1189]
image = Image.open(image_path)
image

In [ ]:
image = np.array(image.convert('RGB'))

In [ ]:
image.shape

In [ ]:
label = np.array(train_df['level2'])

In [ ]:
label.shape

In [ ]:
def get_image(image_path, width, height, mode):
    image = Image.open(image_path)
    image = image.resize([width, height], Image.BILINEAR)
    return np.array(image.convert(mode))

In [ ]:
width, height = 224, 224
mode = 'RGB'
image_files = train_df['path']

data_batch = np.array([get_image(sample_file, width, height, mode) for sample_file in image_files])

In [ ]:
data_batch.shape

In [ ]:
plt.imshow(data_batch[1])

In [ ]:
from PIL import Image

width, height = 224, 224
mode = 'RGB'
    
def get_image(image_path, width, height, mode):
    image = Image.open(image_path)
    image = image.resize([width, height], Image.BILINEAR)
    return np.array(image.convert(mode))


base_image_dir = '../data/MESSIDOR_patches'
retina_df = pd.read_csv(os.path.join(base_image_dir, 'messidor_labels.csv'))
retina_df['level'] = retina_df['level'].map(lambda x: 0 if x == 0 else 1)
retina_df['PatientId'] = retina_df['image'] #map(lambda x: x.split('_')[0])
retina_df['path'] = retina_df['image'].map(lambda x: os.path.join(base_image_dir, '{}.jpg'.format(x)))
retina_df['exists'] = retina_df['path'].map(os.path.exists)
retina_df['level_cat'] = retina_df['level'].map(lambda x: to_categorical(x, 1+retina_df['level'].max()))

retina_df.dropna(inplace=True)  # Remove missing values.
retina_df = retina_df[retina_df['exists']]

rr_df = retina_df[['PatientId', 'level']].drop_duplicates()  # X=PatientId, y=level2
train_ids, valid_ids = train_test_split(rr_df['PatientId'], test_size=0.25, random_state=2018, stratify=rr_df['level'])

train_df = retina_df[retina_df['PatientId'].isin(train_ids)]   
valid_df = retina_df[retina_df['PatientId'].isin(valid_ids)]

X_train = np.array([get_image(sample_file, width, height, mode) for sample_file in train_df['path']])
y_train = np.array(train_df['level'])
X_test = np.array([get_image(sample_file, width, height, mode) for sample_file in valid_df['path']])
y_test = np.array(valid_df['level'])
    

In [ ]:
train_ids.shape, valid_ids.shape

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape